In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cadd_io as cdio
import cadd_main as cdmain
import mdutilities_io as mduio
import mymath as Mmath
import scipy.interpolate as spi
import my_plot as myplot
import cadddatadump as cddump
import cadd_mesh2 as cdmesh
import cadd
import itertools
import os
import sys
import cadd_plot as cdplot
%matplotlib qt
%load_ext autoreload
%autoreload 2

# Preliminaries

## Directories

In [2]:
maindir = 'C:/Users/Varun/Dropbox/Myself/CADD_Code/Tests/Atomistic_Test_2/'
uidir = maindir + 'User Inputs/'
fidir = maindir + 'Fortran Inputs/'
restartdir = maindir + 'Restart Files/'
dumpdir = maindir + 'Dump Files/'
ldumpdir = maindir + 'LAMMPS Style Dump Files/'
lcompdir = maindir + 'LAMMPS Comparison/'

## Simulation Info

In [3]:
simname = 'atomistic_2'
simtype = 'atomistic'

# Construct User Inputs

## Nodes from LAMMPS Dump

### Find File

In [4]:
def init_lammps_filename():
    for filename in os.listdir(lcompdir):
        if filename.endswith('.0.dump'):
            return lcompdir + filename

### Recover xyz

In [5]:
dumpfilenameinit = init_lammps_filename()
dumpfileinit = mduio.DumpFile(dumpfilenameinit)
dumpres = dumpfileinit.read_file(postype='unscaled')
xyz = dumpres[:,2:5]
xyz[:,-1] = 0 # z-coordinate; irrelevant, I suppose
# plt.scatter(xyz[:,0],xyz[:,1])

### Types

In [6]:
natoms = dumpres.shape[0]
types = np.zeros((natoms,3)).astype(int)
fixedflag = 3
freeflag = 0
for i, atominfo in enumerate(dumpres):
    atomtype = atominfo[1]
    if atomtype == 2:
        bcflag = fixedflag
    else:
        bcflag = freeflag
    types[i,:] = [1,1,bcflag]

## Write User Nodes Files

In [7]:
xyfilename = uidir + 'nodes.xy'
typesfilename = uidir + 'nodes.types'
cdio.write_array(xyz,xyfilename)
cdio.write_array(types,typesfilename)

# Read User Inputs

In [9]:
cadddata = cdmain.Simulation(simtype,simname,uidir,fidir,dumpdir,readinput=True)

# Write Fortran Inputs

In [10]:
cadddata.write_fortran_all()

# Plot Dump Files

In [15]:
fig = cadddata.plot_dump_from_increment(fignum=1,increment=5000,style='centro');

# Convert to LAMMPS Dump for Sake of Comparison

In [22]:
def write_lammps_dump(cadddata,increment,pref='lammps_',olddir=dumpdir,newdir=ldumpdir):
    dumpfilename = cadddata.dump_file_name(increment)
    dumpdict = cdio.read_input(dumpfilename,subdir=olddir)
    newfilename = newdir + pref + dumpfilename
    cdio.writeLammpsDump(dumpdict,newfilename,increment)

In [23]:
for increment in range(100,5100,100):
    write_lammps_dump(cadddata,increment)

# Equivalent Files (Restart)

In [25]:
cdio.are_all_files_equivalent(fidir,restartdir)

True

# Compare LAMMPS Dump

In [26]:
def lammps_file_name(lammpssimname,increment):
    return '{}.{}.dump'.format(lammpssimname,increment)

In [27]:
lammpssimname = 'K_CADD_test_lammps_damped'

In [53]:
# sort based on 0
increment = 0
lammpsfile = lammps_file_name(lammpssimname,increment)
lammpsfileobj = mduio.DumpFile(lammpsfile,subdir=lcompdir)
res0 = lammpsfileobj.read_file(postype='unscaled')
idxsort0 = np.argsort(res0[:,0])

None


In [54]:
increment = 5000
lammpsfile = lammps_file_name(lammpssimname,increment)
lammpsfileobj = mduio.DumpFile(lammpsfile,subdir=lcompdir)
res = lammpsfileobj.read_file(postype='unscaled')
idxsort = np.argsort(res[:,0])
xylammps = res[idxsort,2:4]

None


In [55]:
filepath = cadddata.dump_file_path(increment=increment)
dumpdict = cdio.read_from_dump(filepath) # read in dump dictionary from file
cadddump = cddump.CADDDataDump(dumpdict) # create cadddump instance
xyme = cadddump.deformedpos[idxsort0,:]

In [56]:
diff = xylammps - xyme
np.max(np.linalg.norm(diff,axis=1))

8.9223345502613874e-05